In [58]:
from gerrychain import Graph, Election, updaters, GeographicPartition
import numpy as np
import statistics

In [59]:
g = Graph.from_file("./NY/NY.shp")

NUM_DISTRICTS = 27

/Users/rishab/Library/Python/3.9/lib/python/site-packages/gerrychain/graph/graph.py:274: UserWarning: Found islands (degree-0 nodes). Indices of islands: {10308}
  warnings.warn(
/Users/rishab/Library/Python/3.9/lib/python/site-packages/gerrychain/graph/graph.py:179: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


In [60]:
elections = [
    Election("SEN18", {"Democratic": "G18SEND", "Republican": "G18SENR"}),
    Election("GOV18", {"Democratic": "G18GOVD", "Republican": "G18GOVR"}),
    Election("COM18", {"Democratic": "G18COMD", "Republican": "G18COMR"}),
    Election("ATG18", {"Democratic": "G18ATGD", "Republican": "G18ATGR"}),
]
my_updaters = {"population": updaters.Tally("TOTPOP", alias="population")}
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)


In [61]:
initial_partition = GeographicPartition(g,
                                        assignment= "CD", updaters=my_updaters)

In [62]:
def mm(part, election, party):
    if party == "Democratic":
    # Get the vote totals for the Democratic and Republican parties
        votes = part[election].percents("Democratic")
    else:
        votes = part[election].percents("Republican")

    # # Calculate the mean-median difference
    mean_median_diff = statistics.median(votes) - statistics.mean(votes)

    return mean_median_diff

In [63]:
for election in elections:
    print("---------")
    print("election -> {}".format(election.name))
    print("---------")
    for party in ["Democratic", "Republican"]:
        print("party -> {}".format(party))
        diff = mm(initial_partition, election.name, party)
        print("MM Difference -> {}".format(diff))
        print("---------")

    print("\n")

---------
election -> SEN18
---------
party -> Democratic
MM Difference -> -0.0002932774860613385
---------
party -> Republican
MM Difference -> 0.0002932774860613385
---------


---------
election -> GOV18
---------
party -> Democratic
MM Difference -> 0.10609340012018043
---------
party -> Republican
MM Difference -> -0.10609340012018051
---------


---------
election -> COM18
---------
party -> Democratic
MM Difference -> -0.037452158962881836
---------
party -> Republican
MM Difference -> 0.03745215896288179
---------


---------
election -> ATG18
---------
party -> Democratic
MM Difference -> -0.010488808834372015
---------
party -> Republican
MM Difference -> 0.010488808834372088
---------




In [64]:
# Efficiency Gap

def eg(part, election):
    # Get the vote totals for the Democratic and Republican parties
    dem_votes = list(part[election].totals_for_party['Democratic'].values())
    rep_votes = list(part[election].totals_for_party['Republican'].values())

    winning_areas = set()

    # Get winning areas
    for i in range(len(dem_votes)):
        if dem_votes[i] >= rep_votes[i]:
            winning_areas.add(i)

    # Calculate the number of wasted votes for the Democratic and Republican parties
    dem_wasted_votes = 0
    rep_wasted_votes = 0

    for i in range(len(dem_votes)):
        if i in winning_areas:
            dem_wasted_votes += dem_votes[i] - (0.5 * (dem_votes[i] + rep_votes[i]))
            rep_wasted_votes += rep_votes[i]
        else:
            dem_wasted_votes += dem_votes[i]
            rep_wasted_votes += rep_votes[i] - (0.5 * (dem_votes[i] + rep_votes[i]))

    # Calculate the efficiency gap
    efficiency_gap = (rep_wasted_votes - dem_wasted_votes) / sum(dem_votes + rep_votes)

    return efficiency_gap


In [65]:
for election in elections:
    print("---------")
    print("election -> {}".format(election.name))
    print("---------")
    diff = eg(initial_partition, election.name)
    print("Efficiency Gap -> {}".format(diff))
    print("---------")

    print("\n")

---------
election -> SEN18
---------
Efficiency Gap -> -0.17303114929435373
---------


---------
election -> GOV18
---------
Efficiency Gap -> -0.24049997188727895
---------


---------
election -> COM18
---------
Efficiency Gap -> 0.44429081840488993
---------


---------
election -> ATG18
---------
Efficiency Gap -> 0.43999133875317437
---------




In [66]:
## Cut Edges
from gerrychain.tree import recursive_tree_part

In [67]:
total_population = sum([g.nodes[node]["TOTPOP"] for node in g.nodes()])
print("Total Population:", total_population)

Total Population: 20201249


In [68]:
districting_plan = recursive_tree_part(g, range(NUM_DISTRICTS), total_population/NUM_DISTRICTS, "TOTPOP", 0.02, 10)

cut_edges = len([edge for edge in g.edges() if districting_plan[edge[0]] != districting_plan[edge[1]]])
print("Cut edges:", cut_edges)

Cut edges: 2680


In [103]:
from collections import defaultdict

democrat_winning_info = defaultdict(list)
republican_winning_info = defaultdict(list)
## Election: [districts won]

for election in elections:
    # election.name
    dem_votes = initial_partition[election.name].percents("Democratic")
    for district_id, district_votes in enumerate(dem_votes):
        if district_votes > 0.5:
            ## QUESTION: IS THIS SUPPOSED TO BE 0.5?
            ## ARE WE CONSIDERING 50% AS WINNING?
            democrat_winning_info[election.name].append(district_id)
        else:
            republican_winning_info[election.name].append(district_id)

In [104]:
print(democrat_winning_info)

defaultdict(<class 'list'>, {'SEN18': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26], 'GOV18': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 17, 19, 21, 22, 23, 25]})
